In [1]:
# Basic packages
import functools
import operator
import os
import getpass
from typing import Annotated, Sequence, TypedDict

# Langchain packages
from langchain import hub
from langchain.agents import (AgentExecutor,
                              AgentOutputParser,
                              AgentType,
                              Tool,
                              create_react_agent)

from langchain.chains import LLMChain
from langchain_core.messages import (AIMessage,
                                     BaseMessage,
                                     ChatMessage,
                                     FunctionMessage,
                                     HumanMessage,
                                     SystemMessage)
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import (PromptTemplate,
                               ChatPromptTemplate, 
                               HumanMessagePromptTemplate,
                               MessagesPlaceholder)

# Langgraph
from langgraph.graph import END, StateGraph

# Tools
from langchain.tools import BaseTool, StructuredTool, tool
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_experimental.utilities import PythonREPL


def init_llm_langsmith(llm_key = 3):
    """Initialize the LLM model for the LangSmith system."""
    # Set environment variables
    def _set_if_undefined(var: str):
        if not os.environ.get(var):
            os.environ[var] = getpass(f"Please provide your {var}")
    _set_if_undefined("OPENAI_API_KEY")
    _set_if_undefined("LANGCHAIN_API_KEY")
    # Add tracing in LangSmith.
    os.environ["LANGCHAIN_TRACING_V2"] = "true"

    if llm_key == 3:
        llm = "gpt-3.5-turbo-0125"
        os.environ["LANGCHAIN_PROJECT"] = "GPT-3.5 Teaching Agent System TEST 1"
    elif llm_key == 4:
        llm = "gpt-4-0125-preview"
        os.environ["LANGCHAIN_PROJECT"] = "GPT-4 Teaching Agent System TEST 1"
    return llm

llm_ver = init_llm_langsmith(llm_key=3)
llm_model = ChatOpenAI(model_name=llm_ver, temperature=0)

In [ ]:
# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [ ]:
def agent_node(self, state, agent, name):
    """Helper function to create a node for a given agent. Node that invokes agent."""
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

In [ ]:
def create_frontline_node(self, state: BaseMessage):
    """Create the frontline node."""
    system_message = """You are frontline teaching assistant.
                    Be concise in your responses.
                    You can chat with students and help them with basic questions, but if the student has a teaching-related question,
                    do not try to answer the question directly or gather information.
                    Instead, immediately transfer them to the teaching agent system by asking the user to hold for a moment.
                    Otherwise, just respond conversationally."""
    prompt = ChatPromptTemplate.from_messages([("system", system_message),
                                                MessagesPlaceholder(variable_name="messages")])
    chain = prompt | self.model
    node = functools.partial(self.agent_node, agent=chain, name="Frontline")
    return node

In [ ]:
"""Teaching Agent System Graph."""
graph = StateGraph(AgentState)

frontline_node = create_frontline_node()
graph.add_node(frontline_node)
graph.set_entry_point(frontline_node)





## Define Tools

In [24]:
"""Tools."""
search = DuckDuckGoSearchAPIWrapper()
search_tool = Tool(name="Current Search",
                   func=search.run,
                   description="Useful when you need to answer questions about nouns, current events or the current state of the world."
                   )

# Gradient descent code
def coding_function(query: str):
    return """This is a coding example.

    ```python
    def gradient_descent(x, y, learning_rate=0.01, epochs=100):
        m = 0
        b = 0
        n = len(x)
        for _ in range(epochs):
            y_pred = m * x + b
            m_gradient = (-2/n) * sum(x * (y - y_pred))
            b_gradient = (-2/n) * sum(y - y_pred)
            m -= learning_rate * m_gradient
            b -= learning_rate * b_gradient
        return m, b
    ```
    """

coding_tool = StructuredTool.from_function(
                            func=coding_function,
                            name="Coding Tool",
                            description="Useful when you need to answer questions using a coded example."
                            )


tools = [search_tool, coding_tool]

# Running Teaching Agents System with Memory

In [27]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
SYSTEM_MESSAGE = """Teaching Assistant is designed to be able to assist with teaching related tasks, from answering simple questions to providing in-depth explanations and discussions on STEM topics. 
As a language model, Teaching Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Teaching Assistant is constantly learning and improving, and its capabilities are constantly evolving. 
It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. 
Additionally, Teaching Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Teaching Assistant is a powerful tool that can help students with understanding STEM topics. 
Whether you need help with a specific question or just want to have a conversation about a particular topic, Teaching Assistant is here to assist.
"""

COURSE = "Introduction to Computer Science"
SUBJECT = "Gradient Descent"

prompt_hub_template = hub.pull("augustsemrau/react-teaching-chat").template
prompt_template = PromptTemplate.from_template(template=prompt_hub_template)
prompt = prompt_template.partial(system_message=SYSTEM_MESSAGE, course_name=COURSE, subject_name=SUBJECT)
print(prompt)

agent = create_react_agent(llm=llm_model, tools=tools, prompt=prompt, output_parser=None)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=False, handle_parsing_errors=True)


input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] partial_variables={'system_message': 'Teaching Assistant is designed to be able to assist with teaching related tasks, from answering simple questions to providing in-depth explanations and discussions on STEM topics. \nAs a language model, Teaching Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nTeaching Assistant is constantly learning and improving, and its capabilities are constantly evolving. \nIt is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. \nAdditionally, Teaching Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a w

In [28]:
agent_executor.invoke({"input": "Hello I am August?",})["output"]

'Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?'

In [29]:
agent_executor.invoke({"input": "What is it about?",})["output"]

'Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving towards the minimum of the function. It is commonly used in training models to find the optimal parameters that best fit the data.'

In [17]:
agent_executor.invoke({"input": "Can you give an analogy for that?",})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Sure! An analogy for gradient descent is like trying to find the lowest point in a hilly terrain by taking small steps downhill in the steepest direction. You keep adjusting your steps until you reach the lowest point.

> Finished chain.


{'input': 'Can you give an analogy for that?',
 'chat_history': [HumanMessage(content='Hello I am August?'),
  AIMessage(content='Hello August! How can I assist you today?'),
  HumanMessage(content='I want to know what gradient descent is'),
  AIMessage(content='Gradient descent is an optimization algorithm used in machine learning to minimize the loss function by iteratively moving in the direction of the steepest descent of the loss function. It is commonly used in training machine learning models, such as neural networks, to find the optimal parameters that minimize the error between the predicted output and the actual output.'),
  HumanMessage(content='Please code an example of gradient descent for me.'),
  AIMessage(content='Here is an example of gradient descent algorithm:\n\n```python\n# Initialize variables\nx = 5\ny = 10\n\n# Calculate the sum\nz = x + y\n\n# Print the result\nprint(z)\n```'),
  HumanMessage(content='Can you give an analogy for that?'),
  AIMessage(content='Su

In [30]:
agent_executor.invoke({"input": "Please code an example of gradient descent for me.",})

{'input': 'Please code an example of gradient descent for me.',
 'chat_history': [HumanMessage(content='Hello I am August?'),
  AIMessage(content='Hello August! How can I assist you today with learning about Gradient Descent in Computer Science?'),
  HumanMessage(content='What is it about?'),
  AIMessage(content='Gradient Descent is an optimization algorithm used in machine learning and deep learning to minimize the loss function by iteratively moving towards the minimum of the function. It is commonly used in training models to find the optimal parameters that best fit the data.'),
  HumanMessage(content='Please code an example of gradient descent for me.'),
  AIMessage(content='Here is an example of gradient descent in Python:\n\n```python\ndef gradient_descent(x, y, learning_rate=0.01, epochs=100):\n    m = 0\n    b = 0\n    n = len(x)\n    for _ in range(epochs):\n        y_pred = m * x + b\n        m_gradient = (-2/n) * sum(x * (y - y_pred))\n        b_gradient = (-2/n) * sum(y - 

# Leftover Code


In [ ]:
search = DuckDuckGoSearchAPIWrapper()

search_tool = Tool(name="Current Search",
                   func=search.run,
                   description="Useful when you need to answer questions about nouns, current events or the current state of the world."
                   )

tools = [search_tool]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot teaching assistant having a conversation with a human."),
            # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"),
            # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"),
            # Where the human input will injected
    ]
)

llm = ChatOpenAI()

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

chat_llm_chain.predict(human_input="Hi there my name is August Semrau.")